In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [2]:
# 如果是linux系统
trainPath = os.getcwd()+'/train.csv'
testPath = os.getcwd()+'/test.csv'
train_df = pd.read_csv(trainPath, index_col=0)
test_df = pd.read_csv(testPath, index_col=0)

/Users/xhxt2008/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# 如果是win系统

#train_df = pd.read_csv('train.csv', index_col=0)
#test_df = pd.read_csv('test.csv', index_col=0)

In [4]:
#去除y值
y_train = train_df.pop('acc_now_delinq')

In [57]:
tempData = train_df.join(y_train)
tempData.corr().acc_now_delinq.sort_values(ascending=False)

acc_now_delinq                 1.000000
total_acc                      0.027241
int_rate                       0.027073
tot_cur_bal                    0.023586
collections_12_mths_ex_med     0.020847
annual_inc                     0.015529
total_cu_tl                    0.015045
out_prncp                      0.013840
out_prncp_inv                  0.013830
mths_since_rcnt_il             0.012579
total_bal_il                   0.012246
total_rev_hi_lim               0.009179
installment                    0.007723
funded_amnt_inv                0.005468
funded_amnt                    0.005268
loan_amnt                      0.005155
dti                            0.003400
total_rec_late_fee             0.003357
total_rec_int                  0.002578
recoveries                     0.000475
collection_recovery_fee        0.000203
tot_coll_amt                   0.000083
pub_rec                        0.000069
revol_bal                     -0.000857
open_rv_12m                   -0.001411


In [5]:
#合并训练、测试集
all_df = pd.concat((train_df, test_df), axis = 0)

In [93]:
all_df.shape

(887379, 68)

In [6]:
data = all_df

In [7]:
original_df = all_df.copy()

In [8]:
all_df.columns.values

array(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'earliest_cr_line', 'mths_since_last_record',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m'

In [16]:
data = original_df

In [9]:
# 贷款状态只保留good 和bad: loan_status->loan_condition
bad_loan = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "In Grace Period", 
            "Late (16-30 days)", "Late (31-120 days)"]


data['loan_condition'] = np.nan

def loan_condition(status):
    if status in bad_loan:
        return 'Bad Loan'
    else:
        return 'Good Loan'
    
    
data['loan_condition'] = data['loan_status'].apply(loan_condition)

In [10]:
#  地域划分addr_state->region
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

data['region'] = np.nan

def finding_regions(state):
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    
data['region'] = data['addr_state'].apply(finding_regions)

In [11]:
#sub_grade

sub_grade_mapping = {'A1':0,'A2':1,'A3':2,'A4':3,'A5':4,
                     'B1':5,'B2':6,'B3':7,'B4':8,'B5':9,
                     'C1':10,'C2':11,'C3':12,'C4':13,'C5':14,
                     'D1':15,'D2':16,'D3':17,'D4':18,'D5':19,
                     'E1':20,'E2':21,'E3':22,'E4':23,'E5':24,
                     'F1':25,'F2':26,'F3':27,'F4':28,'F5':29,
                     'G1':30,'G2':31,'G3':32,'G4':33,'G5':34}

data.sub_grade = data.sub_grade.map(sub_grade_mapping)



In [12]:
data.issue_d = pd.to_datetime(data['issue_d'])
data.earliest_cr_line = pd.to_datetime(data['earliest_cr_line'])

In [13]:
data['issue_d_year']=data.issue_d.dt.year
data['issue_d_month'] = data.issue_d.dt.month
data['earliest_cr_line_year']=data.earliest_cr_line.dt.year
data['earliest_cr_line_month'] = data.earliest_cr_line.dt.month

## 要删除的

In [15]:
'''
for delete: grade,zip_code,inq_last_12m,total_cu_tl,open_acc_6m,open_il_12m,open_il_24m,open_il_6m,total_bal_il,
open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,policy_code,emp_title,emp_title,desc,title,zip_code,dti_joint,
annual_inc_joint,varification_status_joint,il_uti,mths_since_last_record,mths_since_last_major_derog,addr_state,
issue_d,earliest_cr_line
新增：loan_status,addr_state,application_type,pymnt_plan
'''
feature_delete = {'grade','zip_code','inq_last_12m','total_cu_tl','open_acc_6m','open_il_12m','open_il_24m',
                  'open_il_6m','total_bal_il','open_rv_24m','open_rv_12m','max_bal_bc','all_util','inq_fi',
                 'emp_title','policy_code','desc','title','dti_joint','annual_inc_joint','addr_state','mths_since_rcnt_il',
                 'verification_status_joint','il_util','mths_since_last_record','mths_since_last_major_derog',
                  'issue_d','earliest_cr_line','loan_status','addr_state','application_type','pymnt_plan'}
data = data.drop(feature_delete,axis=1)

In [25]:
data.loan_status

AttributeError: 'DataFrame' object has no attribute 'loan_status'

## 二值化

In [72]:
#for binarilization：pymnt_plan,term, application_type, initial_list_status
#减少：pymnt_plan，application_type  增加：loan_condition,region
feature_for_binary = {'term','initial_list_status','loan_condition','region'}
def binary(feature_for_binary,data):
    for items in feature_for_binary: 
        mapping = {label:idx for idx,label in enumerate(set(data[items]))}  
        data[items] = data[items].map(mapping) 
    return data

In [73]:
binary(feature_for_binary,data)
temp = data.copy()

In [18]:
#emp_length 不再需要fillna！
emp_length_mapping = {'10+ years':10,'9 years':9,'8 years':8,'7 years':7,'6 years':6,
                       '5 years':5,'4 years':4,'3 years':3,'2 years':2,'1 year':1,'< 1 year':0.5,'n/a':0}
data.emp_length = data.emp_length.map(emp_length_mapping) 

## 填充null值

In [19]:
'''
少量null值的处理，用均值替代：emp_length, revol_util, annual_inc, total_acc, earliest_cr_line, tot_cur_bal, tot_coll_amt
total_rev_hi_lim, revol_util, pub_rec, collections_12_mths_ex_med
减少：emp_length
'''
feature_fill_null = {'revol_util','annual_inc','total_acc','earliest_cr_line_year','earliest_cr_line_month','tot_cur_bal',
                    'tot_coll_amt','total_rev_hi_lim','revol_util','pub_rec','collections_12_mths_ex_med'}
def fill_null(feature_fill_null,data):
    for items in feature_fill_null:
        try:
            time.strptime(data[items].mode()[0],"%b-%Y")
            data[items] = data[items].fillna(data[items].mode()[0])
            print (data[items].mode()[0])
#            print ('yes')
        except:
            data[items] = data[items].fillna(data[items].mean())
            print (data[items].mean())
#            print('no')
#         if time.strptime(feature_dropna.mode()[0],"%b-%Y"):
#             data[items] = data[items].fillna(feature_dropna.mode())
#             print feature_dropna.mode()
#         else:
#             data[items] = data[items].fillna(feature_dropna.mean())
    return data

In [20]:
fill_null(feature_fill_null,data)

6.75610976503
139458.189336
55.0676932314
225.702610319
75027.5877608
1997.70018144
25.2680261453
32068.6200454
0.0143795210734
0.195307375895


,loan_amnt,term,int_rate,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_condition,region,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month
member_id,,,,,,,,,,,,,,,,,,,,,
13599771,8875.0,0,18.25,17,8.0,MORTGAGE,55000.0,Verified,debt_consolidation,23.65,...,0.0,0.00000,144784.000000,16700.000000,Bad Loan,SouthWest,2014,3,1985.0,2.0
1491956,12500.0,0,8.90,4,1.0,MORTGAGE,43000.0,Source Verified,debt_consolidation,10.58,...,0.0,225.70261,139458.189336,32068.620045,Good Loan,MidWest,2012,5,1992.0,7.0
64687987,33600.0,1,18.55,21,9.0,RENT,101300.0,Source Verified,debt_consolidation,20.00,...,0.0,770.00000,106215.000000,41300.000000,Good Loan,NorthEast,2015,9,2001.0,5.0
8446850,17000.0,1,9.71,5,10.0,MORTGAGE,47840.0,Verified,debt_consolidation,4.01,...,0.0,102.00000,129416.000000,22800.000000,Good Loan,SouthWest,2013,8,1988.0,9.0
58471388,14000.0,1,16.99,17,5.0,RENT,30000.0,Source Verified,debt_consolidation,31.85,...,0.0,0.00000,15589.000000,10300.000000,Bad Loan,NorthEast,2015,7,1999.0,3.0
5337525,24000.0,1,17.27,14,10.0,OWN,70000.0,Verified,debt_consolidation,33.54,...,0.0,0.00000,311770.000000,24600.000000,Good Loan,SouthEast,2013,4,1983.0,4.0
30165519,12000.0,1,16.29,16,1.0,MORTGAGE,40000.0,Source Verified,credit_card,8.49,...,0.0,1380.00000,10992.000000,8900.000000,Good Loan,SouthWest,2014,10,2006.0,3.0
1909391,23325.0,1,20.49,20,4.0,MORTGAGE,110000.0,Source Verified,debt_consolidation,20.11,...,0.0,0.00000,193824.000000,14800.000000,Bad Loan,SouthEast,2012,11,1998.0,11.0
43182363,15000.0,1,10.49,7,3.0,MORTGAGE,100000.0,Not Verified,credit_card,9.89,...,0.0,0.00000,241273.000000,36000.000000,Good Loan,NorthEast,2015,1,1996.0,8.0


In [21]:
#fill null = 0: collections_12_mths_ex_med
data.collections_12_mths_ex_med = data.collections_12_mths_ex_med.fillna(0)

In [28]:
temp1 = data.copy()

## 离散多类型变量

In [30]:
#for get_dummies: home_ownership, verification_status, loan_status,purpose
#减少：loan_status

feature_for_dummy = {'home_ownership','verification_status','purpose'}
def multi_get_dummies(feature_for_dummy,data):
    for items in feature_for_dummy: 
        data = data.join(pd.get_dummies(data[items],prefix=items))
        data.pop(items)
    return data    

In [21]:
# loan_status_mapping = {'Charged Off':'CF','Fully Paid':'FP','Current':'Current','In Grace Period':'IGP','Late (31-120 days)':'L31-120',
#                        'Issued':'Issued','Does not meet the credit policy. Status:Charged Off':'NotMeetCF','Does not meet the credit policy. Status:Fully Paid':'NotMeetFP',
#                        'Default':'Default','Late (16-30 days)':'L16-30'}
# data.loan_status = data.loan_status.map(loan_status_mapping) 

In [22]:
# data.loan_status.unique()

array(['CF', 'FP', 'Current', 'IGP', 'L31-120', 'Issued', 'NotMeetCF',
       'NotMeetFP', 'Default', 'L16-30'], dtype=object)

In [31]:
data = multi_get_dummies(feature_for_dummy,data)

In [39]:
data.columns.values

array(['loan_amnt', 'term', 'int_rate', 'sub_grade', 'emp_length',
       'annual_inc', 'dti', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'loan_condition',
       'region', 'issue_d_year', 'issue_d_month', 'earliest_cr_line_year',
       'earliest_cr_line_month', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy',

In [84]:
high_corr = {'int_rate','out_prncp_inv','total_pymnt_inv','total_pymnt','funded_amnt','funded_amnt_inv','installment'}
data = data.drop(high_corr,axis=1)

In [81]:
data_corr.purpose_car.sort_values(ascending=False)

purpose_car                            1.000000
verification_status_Not Verified       0.032989
initial_list_status                    0.020818
earliest_cr_line_year                  0.013752
home_ownership_RENT                    0.012604
home_ownership_OWN                     0.008530
loan_condition                         0.003313
collections_12_mths_ex_med             0.001911
home_ownership_OTHER                   0.000936
tot_coll_amt                           0.000420
home_ownership_ANY                    -0.000185
earliest_cr_line_month                -0.000414
total_rec_late_fee                    -0.000448
home_ownership_NONE                   -0.000754
pub_rec                               -0.001928
purpose_educational                   -0.002193
purpose_renewable_energy              -0.002558
issue_d_month                         -0.002756
collection_recovery_fee               -0.003662
recoveries                            -0.004678
purpose_wedding                       -0

In [86]:
data.hcolumns

AttributeError: 'DataFrame' object has no attribute 'hcolumns'

In [25]:
data.isnull().sum()

loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term                                   0
int_rate                               0
installment                            0
sub_grade                              0
emp_length                             0
annual_inc                             0
pymnt_plan                             0
dti                                    0
pub_rec                                0
revol_bal                              0
revol_util                             0
total_acc                              0
initial_list_status                    0
out_prncp                              0
out_prncp_inv                          0
total_pymnt                            0
total_pymnt_inv                        0
total_rec_prncp                        0
total_rec_int                          0
total_rec_late_fee                     0
recoveries                             0
collection_recov

In [87]:
processed_train_df = data.loc[train_df.index]
processed_test_df = data.loc[test_df.index]

In [88]:
processed_train_df.shape

(709903, 50)

In [89]:
processed_test_df.shape

(177476, 50)

In [90]:
processed_train_df2 = processed_train_df.join(y_train)

In [91]:
processed_train_df2.shape

(709903, 51)

In [92]:
processed_train_df2.to_csv("processed_train2.csv" , encoding = "utf-8")
processed_test_df.to_csv("processed_test2.csv" , encoding = "utf-8")

In [23]:
from sklearn import cross_validation

/Users/xhxt2008/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
processed_train_df = pd.read_csv(os.getcwd()+'/processed_train.csv', index_col=0)
processed_test_df = pd.read_csv(os.getcwd()+'/processed_train.csv', index_col=0)

In [28]:
y_train = processed_train_df.pop('acc_now_delinq')

In [29]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(processed_train_df, y_train, 
                                                                     test_size=0.37, random_state=0)

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [20]:
result = knn.predict(X_test)

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rf = ("RandomForestClassifier, max_features='sqrt'",
        RandomForestClassifier(warm_start=True, oob_score=True,
                               max_features="sqrt",n_estimators=100,
                               random_state=123))

In [35]:
rf[1].fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=True, random_state=123,
            verbose=0, warm_start=True)

In [36]:
pre = rf[1].predict(X_test)

In [38]:
pre.sum()

0

In [7]:
data.out_prncp.corr(data.out_prncp_inv)

0.9999972465636372

In [13]:
data.sub_grade.skew()

0.57094469868205833

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=True, random_state=123,
            verbose=0, warm_start=True)